In [2]:
import sys
import os
sys.path.append(os.path.abspath('../../'))
import torch
import pandas as pd
import numpy as np
import wandb
import matplotlib.pyplot as plt
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader
from random import randint
from data_analysis.functions import GetCSVData, PlotRecordData

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
TRAIN_TXT_PATH = "../train_csv_locations.txt"
TEST_TXT_PATH = "../test_csv_locations.txt"
TIME_COL = "Time (sec)"

WANDB_PROJECT_NAME = "Classifier"
MODELS_PATH = "Classifiers"

EPOCHS = 1000

In [5]:
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)

In [32]:
class ClassifierAcceleratorDataset(Dataset):
    def __init__(self, csv_files):
        """
        Args:
            csv_file (string): Paths to .txt which contains all recordings paths.
        """
        self.files = open(csv_files,'r').read().split('\n')
        self.n_files = len(self.files)

    def __len__(self):
        return self.n_files

    def __getitem__(self, idx, as_df=False):
        path, record_class = self.files[idx].split('\t')
        class_to_ret = np.array([float(record_class)])
        df = GetCSVData(path)
        if as_df:
            return df,class_to_ret
        return df.to_numpy()[:,:-1],class_to_ret


In [34]:
class LinearClassifier(torch.nn.Module):
	def __init__(self):
		super().__init__()
		
		self.seq = torch.nn.Sequential(
			torch.nn.Linear(677*3, 1024),
			torch.nn.ReLU(),
			torch.nn.Linear(1024, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 256),
			torch.nn.ReLU(),
			torch.nn.Linear(256, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 64),
			torch.nn.ReLU(),
			torch.nn.Linear(64, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 1),
			torch.nn.Softmax(-1),
		)

	def forward(self, x):
		x = torch.flatten(x, start_dim=1)
		return self.seq(x)

In [35]:
class ConvAutoEncoder(torch.nn.Module):
	def __init__(self):
		super().__init__()

		self.seq = torch.nn.Sequential(
			torch.nn.Conv1d(3, 8,kernel_size=10, stride=5),
			torch.nn.ReLU(),
			torch.nn.Conv1d(8, 16,kernel_size=10, stride=5),
			torch.nn.ReLU(),
			torch.nn.Conv1d(16, 32,kernel_size=10, stride=5),
			torch.nn.ReLU(),
			torch.nn.Conv1d(32, 64,kernel_size=4, stride=1)
		)
		

	def forward(self, x):
		return self.seq(x)

In [36]:
train_dataset = ClassifierAcceleratorDataset(TRAIN_TXT_PATH)
train_dataloader = DataLoader(train_dataset,batch_size=2,shuffle=False)
test_dataset = ClassifierAcceleratorDataset(TEST_TXT_PATH)
test_dataloader = DataLoader(test_dataset,batch_size=4,shuffle=False)

Linear Model Training

In [ ]:
wandb.init(project=WANDB_PROJECT_NAME)

In [43]:
sub_project_path = os.path.join(MODELS_PATH,'LinearModel')
if not os.path.exists(sub_project_path):
    os.makedirs(sub_project_path)
models_path = os.path.join(sub_project_path,'models')
if not os.path.exists(models_path):
    os.makedirs(models_path)

model = LinearClassifier()
loss_func = torch.nn.BCELoss()
optimizer = Adam(model.parameters(),lr=0.00001) #SGD(model.parameters())

In [ ]:
best_val = -1
best_val_idx = -1
for epoch in range(EPOCHS):
    #train
    model.train()
    epoch_loss = []
    n_batches = len(train_dataloader)
    for idx, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        x, y = data
        x = x.to(torch.float32).to(DEVICE)
        y = y.to(torch.float32).to(DEVICE)

        prediction = model(x)
    
        loss = loss_func(prediction,y)
        epoch_loss.append(float(loss))

        loss.backward()
        optimizer.step()

        print("",end='\rEpoch: {}/{} | Batch: {}/{} | loss: {}'.format(epoch,EPOCHS,idx,n_batches,np.mean(epoch_loss)))
    epoch_final_loss = np.mean(epoch_loss)
    del epoch_loss

    #evaluating
    model.eval()
    optimizer.zero_grad()
    validation_loss = []
    for idx, data in enumerate(test_dataloader):
        x,y = data
        x = x.to(torch.float32).to(DEVICE)
        y = y.to(torch.float32).to(DEVICE)

        prediction = model(x)
        
        loss = loss_func(prediction,y)
        validation_loss.append(float(loss))
    validation_final_loss = np.mean(validation_loss)
    del validation_loss
    print('\nEpoch: {}/{} | train_loss: {} | val_loss: {}\n'.format(epoch,EPOCHS,epoch_final_loss,validation_final_loss))

    save_model_at = os.path.join(models_path,'epoch_{}.pt'.format(epoch))
    torch.save(model,save_model_at)

    # wandb.log({
    #     "epoch_loss":epoch_final_loss,
    #     "epoch_validation_loss":validation_final_loss
    # },sync=True)

    if best_val_idx==-1 or best_val>validation_final_loss:
        best_val=validation_final_loss
        best_val_idx=epoch

best_path = os.path.join(models_path,'epoch_{}.pt'.format(best_val_idx))
new_best_path = os.path.join(models_path,'best.pt')
if os.path.exists(new_best_path):
    os.remove(new_best_path)
os.rename(best_path,new_best_path)

Conv Model Training

In [ ]:
 data = torch.permute(data,(0,2,1)).to(torch.float32).to(DEVICE)